In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler,RobustScaler, PowerTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV

from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier,GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier


pd.set_option('display.max_columns', 500)

In [2]:
data = pd.read_csv("train.csv")
print(data.shape)
data.head()


(116058, 29)


,loan_id,source,financial_institution,interest_rate,unpaid_principal_bal,loan_term,origination_date,first_payment_date,loan_to_value,number_of_borrowers,debt_to_income_ratio,borrower_credit_score,loan_purpose,insurance_percent,co-borrower_credit_score,insurance_type,m1,m2,m3,m4,m5,m6,m7,m8,m9,m10,m11,m12,m13
0,268055008619,Z,"Turner, Baldwin and Rhodes",4.250,214000,360,2012-03-01,05/2012,95,1.0,22.0,694.0,C86,30.0,0.0,0.0,0,0,0,0,0,0,1,0,0,0,0,0,1
1,672831657627,Y,"Swanson, Newton and Miller",4.875,144000,360,2012-01-01,03/2012,72,1.0,44.0,697.0,B12,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,1,0,1
2,742515242108,Z,Thornton-Davis,3.250,366000,180,2012-01-01,03/2012,49,1.0,33.0,780.0,B12,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,601385667462,X,OTHER,4.750,135000,360,2012-02-01,04/2012,46,2.0,44.0,633.0,B12,0.0,638.0,0.0,0,0,0,0,0,0,0,0,1,1,1,1,1
4,273870029961,X,OTHER,4.750,124000,360,2012-02-01,04/2012,80,1.0,43.0,681.0,C86,0.0,0.0,0.0,0,1,2,3,4,5,6,7,8,9,10,11,1


In [3]:
data['first_payment_date'] = pd.DatetimeIndex(data['first_payment_date']).month
data['origination_date'] =  pd.DatetimeIndex(data['origination_date']).month
print(data.shape)
data.head()

(116058, 29)


,loan_id,source,financial_institution,interest_rate,unpaid_principal_bal,loan_term,origination_date,first_payment_date,loan_to_value,number_of_borrowers,debt_to_income_ratio,borrower_credit_score,loan_purpose,insurance_percent,co-borrower_credit_score,insurance_type,m1,m2,m3,m4,m5,m6,m7,m8,m9,m10,m11,m12,m13
0,268055008619,Z,"Turner, Baldwin and Rhodes",4.250,214000,360,3,5,95,1.0,22.0,694.0,C86,30.0,0.0,0.0,0,0,0,0,0,0,1,0,0,0,0,0,1
1,672831657627,Y,"Swanson, Newton and Miller",4.875,144000,360,1,3,72,1.0,44.0,697.0,B12,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,1,0,1
2,742515242108,Z,Thornton-Davis,3.250,366000,180,1,3,49,1.0,33.0,780.0,B12,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,601385667462,X,OTHER,4.750,135000,360,2,4,46,2.0,44.0,633.0,B12,0.0,638.0,0.0,0,0,0,0,0,0,0,0,1,1,1,1,1
4,273870029961,X,OTHER,4.750,124000,360,2,4,80,1.0,43.0,681.0,C86,0.0,0.0,0.0,0,1,2,3,4,5,6,7,8,9,10,11,1


In [4]:
sample = pd.read_csv("sample_submission.csv")
sample.head()


,loan_id,m13
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0


In [5]:
test = pd.read_csv("test.csv")
print(test.shape)
test.head()

(35866, 28)


,loan_id,source,financial_institution,interest_rate,unpaid_principal_bal,loan_term,origination_date,first_payment_date,loan_to_value,number_of_borrowers,debt_to_income_ratio,borrower_credit_score,loan_purpose,insurance_percent,co-borrower_credit_score,insurance_type,m1,m2,m3,m4,m5,m6,m7,m8,m9,m10,m11,m12
0,1,Y,Browning-Hart,3.875,417000,360,01/02/12,Apr-12,75,1,20,790,A23,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,X,OTHER,4.500,113000,360,01/02/12,Apr-12,80,2,33,793,C86,0,784,0,0,0,0,0,0,0,0,0,0,0,0,0
2,3,Y,OTHER,4.500,72000,360,01/01/12,Mar-12,75,1,34,710,C86,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,4,X,"Miller, Mcclure and Allen",4.125,123000,180,01/02/12,Apr-12,41,2,24,798,A23,0,813,0,0,0,0,0,0,0,0,0,0,0,0,0
4,5,X,Browning-Hart,3.250,166000,180,01/02/12,Apr-12,53,2,12,767,A23,0,768,0,0,0,0,0,0,0,0,0,0,0,0,0


In [6]:
test['origination_date'] = pd.DatetimeIndex(test['origination_date']).day
first_payment_date_dict = {'Apr-12': 4, 'Mar-12':3, 'May-12':5, 'Feb-12':2} 
test['first_payment_date'] = test['first_payment_date'].map(first_payment_date_dict)
print(test.shape)
test.head()

(35866, 28)


,loan_id,source,financial_institution,interest_rate,unpaid_principal_bal,loan_term,origination_date,first_payment_date,loan_to_value,number_of_borrowers,debt_to_income_ratio,borrower_credit_score,loan_purpose,insurance_percent,co-borrower_credit_score,insurance_type,m1,m2,m3,m4,m5,m6,m7,m8,m9,m10,m11,m12
0,1,Y,Browning-Hart,3.875,417000,360,2,4,75,1,20,790,A23,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,X,OTHER,4.500,113000,360,2,4,80,2,33,793,C86,0,784,0,0,0,0,0,0,0,0,0,0,0,0,0
2,3,Y,OTHER,4.500,72000,360,1,3,75,1,34,710,C86,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,4,X,"Miller, Mcclure and Allen",4.125,123000,180,2,4,41,2,24,798,A23,0,813,0,0,0,0,0,0,0,0,0,0,0,0,0
4,5,X,Browning-Hart,3.250,166000,180,2,4,53,2,12,767,A23,0,768,0,0,0,0,0,0,0,0,0,0,0,0,0


In [7]:
scaler = StandardScaler()
#scaler = RobustScaler()
#scaler = PowerTransformer()

def clean_train_data(data):
    int_cols = data.describe().columns
    all_cols = data.columns
    
    non_int_cols = [i for i in all_cols if i not in int_cols]
    print(non_int_cols)
    #data.source.unique()
    #for i in non_int_cols:
        #tr = data[i]
        #print(tr.unique())
    
    y = data['m13']
    X = data
    X['first_payment_date'] = data['first_payment_date'] - data['origination_date']
    X = X.drop(['m13','origination_date','loan_id'],axis=1)
    
    #X['first_payment_date'] = pd.DatetimeIndex(X['first_payment_date']).month
    #X['origination_date'] =  pd.DatetimeIndex(X['origination_date']).month
    
    X = pd.get_dummies(X,columns=['source', 'financial_institution', 'loan_purpose'])
    
    #mon_dt = pd.DataFrame(mon_diff,columns=[mon_dt])
    
    #X = 
    
    X_cols = X.columns
    df_X = scaler.fit_transform(X)
    df_scaled = pd.DataFrame(df_X,columns=X_cols)
    return df_scaled,y
    #return X,y


def clean_test_data(test):
    int_cols = test.describe().columns
    all_cols = test.columns
    
    non_int_cols = [i for i in all_cols if i not in int_cols]
    print(non_int_cols)
    #data.source.unique()
    #for i in non_int_cols:
        #tr = data[i]
        #print(tr.unique())
    
    #y = data['m13']
    #X = data.drop(['m13'],axis=1)
    #X = data.drop(['origination_date', 'first_payment_date'],axis=1)
    
    #test['origination_date'] = pd.DatetimeIndex(test['origination_date']).month
    #first_payment_date_dict = {'Apr-12': 4, 'Mar-12':3, 'May-12':5, 'Feb-12':2} 
    #test['first_payment_date'] = test['first_payment_date'].map(first_payment_date_dict)
    X = test
    X['first_payment_date'] = test['first_payment_date'] - test['origination_date']
    X = X.drop(['origination_date','loan_id'],axis=1)
    
    X = pd.get_dummies(X,columns=['source', 'financial_institution', 'loan_purpose'])

    X_cols = X.columns
    df_X = scaler.transform(X)
    df_scaled = pd.DataFrame(df_X,columns=X_cols)
    return df_scaled
    #return X
    

In [8]:
X,y = clean_train_data(data)
print(X.shape)
X.head()

#y.value_counts()

['source', 'financial_institution', 'loan_purpose']
(116058, 48)


,interest_rate,unpaid_principal_bal,loan_term,first_payment_date,loan_to_value,number_of_borrowers,debt_to_income_ratio,borrower_credit_score,insurance_percent,co-borrower_credit_score,insurance_type,m1,m2,m3,m4,m5,m6,m7,m8,m9,m10,m11,m12,source_X,source_Y,source_Z,financial_institution_Anderson-Taylor,financial_institution_Browning-Hart,financial_institution_Chapman-Mcmahon,"financial_institution_Cole, Brooks and Vincent",financial_institution_Edwards-Hoffman,"financial_institution_Martinez, Duffy and Bird","financial_institution_Miller, Mcclure and Allen",financial_institution_Nicholson Group,financial_institution_OTHER,financial_institution_Richards-Walters,financial_institution_Richardson Ltd,"financial_institution_Romero, Woods and Johnson","financial_institution_Sanchez, Hays and Wilkerson",financial_institution_Sanchez-Robinson,financial_institution_Suarez Inc,"financial_institution_Swanson, Newton and Miller","financial_institution_Taylor, Hunt and Rodriguez",financial_institution_Thornton-Davis,"financial_institution_Turner, Baldwin and Rhodes",loan_purpose_A23,loan_purpose_B12,loan_purpose_C86
0,0.826517,0.050345,0.754428,-0.026766,1.594300,-1.207529,-0.898419,-1.798764,3.361200,-1.203344,-0.057163,-0.050458,-0.040072,-0.035599,-0.035069,-0.04275,-0.039070,9.863609,-0.042652,-0.041791,-0.044583,-0.043642,-0.045276,-1.106044,-0.691643,2.631390,-0.064646,-0.615031,-0.057917,-0.119796,-0.208992,-0.205783,-0.167247,-0.136275,-0.865414,-0.112833,-0.063971,-0.080649,-0.086048,-0.101912,-0.12516,-0.250914,-0.104723,-0.075106,7.865748,-1.00749,-0.582239,1.764550
1,2.182212,-0.560025,0.754428,-0.026766,0.264177,-1.207529,1.362454,-1.727692,-0.344138,-1.203344,-0.057163,-0.050458,-0.040072,-0.035599,-0.035069,-0.04275,-0.039070,-0.041221,-0.042652,-0.041791,-0.044583,6.095719,-0.045276,-1.106044,1.445832,-0.380027,-0.064646,-0.615031,-0.057917,-0.119796,-0.208992,-0.205783,-0.167247,-0.136275,-0.865414,-0.112833,-0.063971,-0.080649,-0.086048,-0.101912,-0.12516,3.985426,-0.104723,-0.075106,-0.127133,-1.00749,1.717508,-0.566717
2,-1.342595,1.375719,-1.250874,-0.026766,-1.065945,-1.207529,0.232018,0.238642,-0.344138,-1.203344,-0.057163,-0.050458,-0.040072,-0.035599,-0.035069,-0.04275,-0.039070,-0.041221,-0.042652,-0.041791,-0.044583,-0.043642,-0.045276,-1.106044,-0.691643,2.631390,-0.064646,-0.615031,-0.057917,-0.119796,-0.208992,-0.205783,-0.167247,-0.136275,-0.865414,-0.112833,-0.063971,-0.080649,-0.086048,-0.101912,-0.12516,-0.250914,-0.104723,13.314522,-0.127133,-1.00749,1.717508,-0.566717
3,1.911073,-0.638501,0.754428,-0.026766,-1.239439,0.828137,1.362454,-3.243901,-0.344138,0.467052,-0.057163,-0.050458,-0.040072,-0.035599,-0.035069,-0.04275,-0.039070,-0.041221,-0.042652,7.756016,6.692613,6.095719,5.568680,0.904123,-0.691643,-0.380027,-0.064646,-0.615031,-0.057917,-0.119796,-0.208992,-0.205783,-0.167247,-0.136275,1.155516,-0.112833,-0.063971,-0.080649,-0.086048,-0.101912,-0.12516,-0.250914,-0.104723,-0.075106,-0.127133,-1.00749,1.717508,-0.566717
4,1.911073,-0.734416,0.754428,-0.026766,0.726829,-1.207529,1.259687,-2.106744,-0.344138,-1.203344,-0.057163,-0.050458,19.418760,36.047883,48.226620,48.36153,57.069319,59.387759,61.834283,62.340670,60.590177,61.349964,61.708245,0.904123,-0.691643,-0.380027,-0.064646,-0.615031,-0.057917,-0.119796,-0.208992,-0.205783,-0.167247,-0.136275,1.155516,-0.112833,-0.063971,-0.080649,-0.086048,-0.101912,-0.12516,-0.250914,-0.104723,-0.075106,-0.127133,-1.00749,-0.582239,1.764550


In [9]:
test_X = clean_test_data(test)
print(test_X.shape)
test_X.head()

['source', 'financial_institution', 'loan_purpose']
(35866, 48)


,interest_rate,unpaid_principal_bal,loan_term,first_payment_date,loan_to_value,number_of_borrowers,debt_to_income_ratio,borrower_credit_score,insurance_percent,co-borrower_credit_score,insurance_type,m1,m2,m3,m4,m5,m6,m7,m8,m9,m10,m11,m12,source_X,source_Y,source_Z,financial_institution_Anderson-Taylor,financial_institution_Browning-Hart,financial_institution_Chapman-Mcmahon,"financial_institution_Cole, Brooks and Vincent",financial_institution_Edwards-Hoffman,"financial_institution_Martinez, Duffy and Bird","financial_institution_Miller, Mcclure and Allen",financial_institution_Nicholson Group,financial_institution_OTHER,financial_institution_Richards-Walters,financial_institution_Richardson Ltd,"financial_institution_Romero, Woods and Johnson","financial_institution_Sanchez, Hays and Wilkerson",financial_institution_Sanchez-Robinson,financial_institution_Suarez Inc,"financial_institution_Swanson, Newton and Miller","financial_institution_Taylor, Hunt and Rodriguez",financial_institution_Thornton-Davis,"financial_institution_Turner, Baldwin and Rhodes",loan_purpose_A23,loan_purpose_B12,loan_purpose_C86
0,0.013100,1.820417,0.754428,-0.026766,0.437672,-1.207529,-1.103953,0.475550,-0.344138,-1.203344,-0.057163,-0.050458,-0.040072,-0.035599,-0.035069,-0.04275,-0.03907,-0.041221,-0.042652,-0.041791,-0.044583,-0.043642,-0.045276,-1.106044,1.445832,-0.380027,-0.064646,1.625935,-0.057917,-0.119796,-0.208992,-0.205783,-0.167247,-0.136275,-0.865414,-0.112833,-0.063971,-0.080649,-0.086048,-0.101912,-0.12516,-0.250914,-0.104723,-0.075106,-0.127133,0.992566,-0.582239,-0.566717
1,1.368795,-0.830331,0.754428,-0.026766,0.726829,0.828137,0.232018,0.546622,-0.344138,0.849306,-0.057163,-0.050458,-0.040072,-0.035599,-0.035069,-0.04275,-0.03907,-0.041221,-0.042652,-0.041791,-0.044583,-0.043642,-0.045276,0.904123,-0.691643,-0.380027,-0.064646,-0.615031,-0.057917,-0.119796,-0.208992,-0.205783,-0.167247,-0.136275,1.155516,-0.112833,-0.063971,-0.080649,-0.086048,-0.101912,-0.12516,-0.250914,-0.104723,-0.075106,-0.127133,-1.007490,-0.582239,1.764550
2,1.368795,-1.187834,0.754428,-0.026766,0.437672,-1.207529,0.334785,-1.419712,-0.344138,-1.203344,-0.057163,-0.050458,-0.040072,-0.035599,-0.035069,-0.04275,-0.03907,-0.041221,-0.042652,-0.041791,-0.044583,-0.043642,-0.045276,-1.106044,1.445832,-0.380027,-0.064646,-0.615031,-0.057917,-0.119796,-0.208992,-0.205783,-0.167247,-0.136275,1.155516,-0.112833,-0.063971,-0.080649,-0.086048,-0.101912,-0.12516,-0.250914,-0.104723,-0.075106,-0.127133,-1.007490,-0.582239,1.764550
3,0.555378,-0.743136,-1.250874,-0.026766,-1.528596,0.828137,-0.692885,0.665076,-0.344138,0.925233,-0.057163,-0.050458,-0.040072,-0.035599,-0.035069,-0.04275,-0.03907,-0.041221,-0.042652,-0.041791,-0.044583,-0.043642,-0.045276,0.904123,-0.691643,-0.380027,-0.064646,-0.615031,-0.057917,-0.119796,-0.208992,-0.205783,5.979170,-0.136275,-0.865414,-0.112833,-0.063971,-0.080649,-0.086048,-0.101912,-0.12516,-0.250914,-0.104723,-0.075106,-0.127133,0.992566,-0.582239,-0.566717
4,-1.342595,-0.368194,-1.250874,-0.026766,-0.834619,0.828137,-1.926088,-0.069338,-0.344138,0.807415,-0.057163,-0.050458,-0.040072,-0.035599,-0.035069,-0.04275,-0.03907,-0.041221,-0.042652,-0.041791,-0.044583,-0.043642,-0.045276,0.904123,-0.691643,-0.380027,-0.064646,1.625935,-0.057917,-0.119796,-0.208992,-0.205783,-0.167247,-0.136275,-0.865414,-0.112833,-0.063971,-0.080649,-0.086048,-0.101912,-0.12516,-0.250914,-0.104723,-0.075106,-0.127133,0.992566,-0.582239,-0.566717


In [10]:
X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=.20, random_state=99,shuffle=True)

print(X_train.shape)
y_train.value_counts()


(92846, 48)


0    92345
1      501
Name: m13, dtype: int64

In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [12]:
import torch.utils.data as data_utils
from torch import optim

In [13]:
class DelinqueNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(48,16)
        self.fc2 = nn.Linear(16,24)
        self.fc3 = nn.Linear(24,1)
    def forward(self,x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.dropout(x, p=.25)
        x = F.sigmoid(self.fc3(x))
        return x
    
        

In [14]:
model = DelinqueNet().double()

In [15]:
model

DelinqueNet(
  (fc1): Linear(in_features=48, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=24, bias=True)
  (fc3): Linear(in_features=24, out_features=1, bias=True)
)

In [16]:
x_tr = torch.from_numpy(X_train.values).double()
y_tr = torch.from_numpy(y_train.values).double()


In [17]:
y_tr

tensor([0., 0., 0.,  ..., 0., 0., 0.], dtype=torch.float64)

In [18]:
criterion = nn.BCELoss()
optimizer = optim.SGD(model.parameters(),lr=0.001)

In [19]:
epoch = 2
batch = 64

train = data_utils.TensorDataset(x_tr, y_tr)
train_loader = data_utils.DataLoader(train, batch_size=batch, shuffle=True)


In [20]:
x_val = torch.from_numpy(X_test.values).double()
y_val = torch.from_numpy(y_test.values).double()

valid = data_utils.TensorDataset(x_val, y_val)
valid_loader = data_utils.DataLoader(valid, batch_size=batch, shuffle=True)

In [21]:
nxt1 = iter(train)

print(nxt1)

In [22]:
nxt = iter(train_loader)
dft = nxt.next()
print(dft[0].size())

torch.Size([64, 48])


In [23]:
for i in range(epoch):
    running_loss = 0
    for b, data in enumerate(train_loader, 0):
        inputs, labels = data
        y_pred = model(inputs)
        loss = criterion(y_pred, labels)
        
        if b % 10000:
            pass
            #print('Epochs: {}, batch: {} loss: {}'.format(i, b, loss))
        #reset gradients
        optimizer.zero_grad()

        # backward pass
        loss.backward()

        # update weights
        optimizer.step()
        running_loss += loss.item()
    else :
        valid_loss = 0
        accuracy = 0
        model.eval()
        print("Validation started")
        with torch.no_grad():
            for b, val_data in enumerate(valid_loader,0):
                inputs, labels = val_data
                val_pred = model(inputs)
                valid_loss += criterion(val_pred,labels)
                ps = torch.exp(val_pred)
                labels = labels.type(torch.LongTensor)
                
                top_p, top_class = ps.topk(1, dim=1)
                equals = top_class == labels.view(*top_class.shape)
                accuracy += torch.mean(equals.type(torch.FloatTensor))
                
                #if b % 50000 :
                    #print(val_pred.size())
                    #print(top_p, top_class)
                    
        model.train()
        print("Epoch: {}/{}.. ".format(i+1, epoch),
              "Training Loss: {:.3f}.. ".format(running_loss/len(train_loader)),
              "Validation Loss: {:.3f}.. ".format(valid_loss/len(valid_loader)),
              "Validation Accuracy: {:.3f}".format(accuracy/len(valid_loader)))
        

/Users/sivakumarpadala/anaconda3/envs/tensorflow_env/lib/python3.6/site-packages/torch/nn/functional.py:1332: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/Users/sivakumarpadala/anaconda3/envs/tensorflow_env/lib/python3.6/site-packages/torch/nn/functional.py:2016: UserWarning: Using a target size (torch.Size([64])) that is different to the input size (torch.Size([64, 1])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))
/Users/sivakumarpadala/anaconda3/envs/tensorflow_env/lib/python3.6/site-packages/torch/nn/functional.py:2016: UserWarning: Using a target size (torch.Size([46])) that is different to the input size (torch.Size([46, 1])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))


Validation started


/Users/sivakumarpadala/anaconda3/envs/tensorflow_env/lib/python3.6/site-packages/torch/nn/functional.py:2016: UserWarning: Using a target size (torch.Size([44])) that is different to the input size (torch.Size([44, 1])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))


Epoch: 1/2..  Training Loss: 0.524..  Validation Loss: 0.372..  Validation Accuracy: 0.994
Validation started
Epoch: 2/2..  Training Loss: 0.282..  Validation Loss: 0.213..  Validation Accuracy: 0.994


In [24]:
tst_x = torch.from_numpy(test_X.values).double()

#valid = data_utils.TensorDataset(x_val, y_val)
valid_loader = data_utils.DataLoader(tst_x, batch_size=batch, shuffle=False)

In [25]:
lst = []

for k in valid_loader:
    pred = model(k)
    ps = torch.exp(pred)
    top_p, top_class = ps.topk(1, dim=1)
    for j in top_class.numpy():
        lst.append(j)

print(len(lst))
#npp = np.array(lst)

dff = pd.DataFrame(lst,columns=['m13'])
print(dff.m13.value_counts())
dff.tail()


35866
0    35866
Name: m13, dtype: int64


,m13
35861,0
35862,0
35863,0
35864,0
35865,0
